In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, re, os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split
import glob
print("Tensorflow version " + tf.__version__)
import cv2
from cv2 import cvtColor
#from PIL import Image
import numpy.ma as ma
import time
from skimage.transform import resize


import tensorflow_addons as tfa



# Initialization


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Acknowledgment


This notebook was inspiried by 

https://www.kaggle.com/h053473666/0-354-efnb7-classification-weights-0-4-0-6

https://www.kaggle.com/thedrcat/fastai-quick-submission-template

https://www.kaggle.com/linshokaku/faster-hpa-cell-segmentation


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
session = InteractiveSession(config=config)



In [ ]:
IM_SIZE = 256

In [ ]:
specified_class_names = """0. Nucleoplasm
1. Nuclear membrane
2. Nucleoli
3. Nucleoli fibrillar center
4. Nuclear speckles
5. Nuclear bodies
6. Endoplasmic reticulum
7. Golgi apparatus
8. Intermediate filaments
9. Actin filaments 
10. Microtubules
11. Mitotic spindle
12. Centrosome
13. Plasma membrane
14. Mitochondria
15. Aggresome
16. Cytosol
17. Vesicles and punctate cytosolic patterns
18. Negative"""

class_names = [class_name.split('. ')[1] for class_name in specified_class_names.split('\n')]
class_names

In [ ]:
len(class_names) +1 

In [ ]:
df_imlist = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')

In [ ]:


if len(df_imlist) == 559:
    debug = True
    df_imlist = df_imlist[:3]
else:
    debug = False

In [ ]:
def blend2_image(image):
    
    directory = '../input/hpa-single-cell-image-classification/test/'
    
    
    
    blue_image = cv2.imread(directory+image + '_blue' + '.png', cv2.IMREAD_GRAYSCALE )#.resize((size,size))
    red_image = cv2.imread(directory+image + '_red' + '.png', cv2.IMREAD_GRAYSCALE )#.resize((size,size))

    #green_image = Image.open(directory+image + '_green' + '.png' )#.resize((size,size))
    
    green_image = cv2.imread(directory+image + '_green' + '.png', cv2.IMREAD_GRAYSCALE)


    
    ## Convert red 
    
    bild_array_red = np.asarray(red_image)
    bild_array_red = np.require(bild_array_red, dtype='f4', requirements=['O', 'W'])
    bild_array_red.flags['WRITEABLE'] = True
    bild_array_red /= 255

    
    ## Convert blue
    
    bild_array_blue = np.asarray(blue_image)
    
    bild_array_blue = np.require(bild_array_blue, dtype='f4', requirements=['O', 'W'])
    bild_array_blue.flags['WRITEABLE'] = True
    
    bild_array_blue /= 255

      
    bild_array_green = np.asarray(green_image)
    
    bild_array_green = np.require(bild_array_green, dtype='f4', requirements=['O', 'W'])
    bild_array_green.flags['WRITEABLE'] = True
    
    bild_array_green /= 255


    blend = np.stack((bild_array_red, bild_array_green, bild_array_blue), axis=-1)
    #blend = np.stack((bild_array_blue, bild_array_green, bild_array_red), axis=-1) # lie picture

    
    return blend

In [ ]:
#directory = '../input/hpa-single-cell-image-classification/test/'

#blue_image = Image.open(directory+'6220622a-b7e6-47c7-bdfa-2dfe650771b2' + '_green' + '.png' )



#blue_image

In [ ]:
fig = plt.figure(figsize=(30,30),constrained_layout = True)

plt.imshow(blend2_image('d48f8a42-d427-45d4-a0fd-ce119ba654a7'))

In [ ]:
def get_cropped_resized_cell(img):
    #bmask = msk.astype(int)[...,None]
    #masked_img = img * bmask
    true_points = np.argwhere(img)
    top_left = true_points.min(axis=0)
    bottom_right = true_points.max(axis=0)
    cropped_arr = img[top_left[0]:bottom_right[0]+1,top_left[1]:bottom_right[1]+1]
    cropped_arr = cv2.resize(cropped_arr,(IM_SIZE,IM_SIZE))
    return cropped_arr

In [ ]:
bild = blend2_image('d48f8a42-d427-45d4-a0fd-ce119ba654a7')

bild = get_cropped_resized_cell(bild)

plt.imshow(bild)

In [ ]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

# Cell level Models

In [ ]:
models = []



model = keras.models.load_model('../input/multieffe0-rs10-6e/HPA_model_effect0_RS10_6mcepoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout, "macro_f1" : macro_f1})
model1 = keras.models.load_model('../input/multieff0-rs11/HPA_model_effect0_RS11_6mcepoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout, "macro_f1" : macro_f1})
model2 = keras.models.load_model('../input/multidense/HPA_model_effect0_RS11_6mcepoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout, "macro_f1" : macro_f1})
model3 = keras.models.load_model('../input/pubdensenet/HPA_model_dense121_RS14_15mcepoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout, "macro_f1" : macro_f1})
#model4 = keras.models.load_model('../input/modeleffect/HPA_model_eff3_RS35.h5',custom_objects={"FixedDropout": keras.layers.Dropout})
#model5 = keras.models.load_model('../input/dense121-rs65/HPA_model_dens121_RS65.h5',custom_objects={"FixedDropout": keras.layers.Dropout})
#model6 = keras.models.load_model('../input/pubdens121-rs4-6epoch/HPA_model_den121_RS4_6epoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout})
#model7 = keras.models.load_model('../input/pubeffe3rs2/HPA_model_eff3_RS2_pubda.h5',custom_objects={"FixedDropout": keras.layers.Dropout})
#model8 = keras.models.load_model('../input/pubdensrs0/HPA_model_dens121_RS0_5epoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout})
#model9 = keras.models.load_model('../input/pub-effec-rs8-8epoch/HPA_model_effect_RS8_8epoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout})
#model10 = keras.models.load_model('../input/pubeff0rs95ep/HPA_model_effect0_RS9_5epoch.h5',custom_objects={"FixedDropout": keras.layers.Dropout})


models.append(model)
models.append(model1)
models.append(model2)
models.append(model3)
#models.append(model4)
#models.append(model5)
#models.append(model6)
#models.append(model7)
#models.append(model8)
#models.append(model9)
#models.append(model10)



In [ ]:
#enumerate(model.predict(img)[0])

In [ ]:
#liste = sum([model.predict(img)[0] for model in models]) / len(models)

In [ ]:
#liste

In [ ]:
!pip install ../input/pycocotools/pycocotools-2.0.0

In [ ]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib


def encode_binary_mask(mask: np.ndarray) -> t.Text:
    
    
    """Converts a binary mask into OID challenge encoding ascii text."""
    
  # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [ ]:
!pip install ../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master
#!pip install "../input/hpacellsegmentatorraman/HPA-Cell-Segmentation/"

In [ ]:
!pip install ../input/hpapytorchzoozip/pytorch_zoo-master/

In [ ]:
#!pip install ../input/pycocotools/pycocotools-2.0.0

In [ ]:
#    plt.imshow(test_mask)


In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell,label_nuclei

# Faster Cell mask

In [ ]:
import os.path
import urllib
import zipfile

import numpy as np
import scipy.ndimage as ndi
from skimage import filters, measure, segmentation
from skimage.morphology import (binary_erosion, closing, disk,
                                remove_small_holes, remove_small_objects)

HIGH_THRESHOLD = 0.4
LOW_THRESHOLD = HIGH_THRESHOLD - 0.25


def download_with_url(url_string, file_path, unzip=False):
    """Download file with a link."""
    with urllib.request.urlopen(url_string) as response, open(
        file_path, "wb"
    ) as out_file:
        data = response.read()  # a `bytes` object
        out_file.write(data)

    if unzip:
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(os.path.dirname(file_path))


def __fill_holes(image):
    """Fill_holes for labelled image, with a unique number."""
    boundaries = segmentation.find_boundaries(image)
    image = np.multiply(image, np.invert(boundaries))
    image = ndi.binary_fill_holes(image > 0)
    image = ndi.label(image)[0]
    return image





def label_cell2(nuclei_pred, cell_pred):
    """Label the cells and the nuclei.
    Keyword arguments:
    nuclei_pred -- a 3D numpy array of a prediction from a nuclei image.
    cell_pred -- a 3D numpy array of a prediction from a cell image.
    Returns:
    A tuple containing:
    nuclei-label -- A nuclei mask data array.
    cell-label  -- A cell mask data array.
    0's in the data arrays indicate background while a continous
    strech of a specific number indicates the area for a specific
    cell.
    The same value in cell mask and nuclei mask refers to the identical cell.
    NOTE: The nuclei labeling from this function will be sligthly
    different from the values in :func:`label_nuclei` as this version
    will use information from the cell-predictions to make better
    estimates.
    """
    def __wsh(
        mask_img,
        threshold,
        border_img,
        seeds,
        threshold_adjustment=0.35,
        small_object_size_cutoff=10,
    ):
        img_copy = np.copy(mask_img)
        m = seeds * border_img  # * dt
        img_copy[m <= threshold + threshold_adjustment] = 0
        img_copy[m > threshold + threshold_adjustment] = 1
        img_copy = img_copy.astype(np.bool)
        img_copy = remove_small_objects(img_copy, small_object_size_cutoff).astype(
            np.uint8
        )

        mask_img[mask_img <= threshold] = 0
        mask_img[mask_img > threshold] = 1
        mask_img = mask_img.astype(np.bool)
        mask_img = remove_small_holes(mask_img, 63)
        mask_img = remove_small_objects(mask_img, 1).astype(np.uint8)
        markers = ndi.label(img_copy, output=np.uint32)[0]
        labeled_array = segmentation.watershed(
            mask_img, markers, mask=mask_img, watershed_line=True
        )
        return labeled_array

    nuclei_label = __wsh(
        nuclei_pred[..., 2] / 255.0,
        0.4,
        1 - (nuclei_pred[..., 1] + cell_pred[..., 1]) / 255.0 > 0.05,
        nuclei_pred[..., 2] / 255,
        threshold_adjustment=-0.25,
        small_object_size_cutoff=32,
    )

    # for hpa_image, to remove the small pseduo nuclei
    nuclei_label = remove_small_objects(nuclei_label, 157)
    nuclei_label = measure.label(nuclei_label)
    # this is to remove the cell borders' signal from cell mask.
    # could use np.logical_and with some revision, to replace this func.
    # Tuned for segmentation hpa images
    threshold_value = max(0.22, filters.threshold_otsu(cell_pred[..., 2] / 255) * 0.5)
    # exclude the green area first
    cell_region = np.multiply(
        cell_pred[..., 2] / 255 > threshold_value,
        np.invert(np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8)),
    )
    sk = np.asarray(cell_region, dtype=np.int8)
    distance = np.clip(cell_pred[..., 2], 255 * threshold_value, cell_pred[..., 2])
    cell_label = segmentation.watershed(-distance, nuclei_label, mask=sk)
    cell_label = remove_small_objects(cell_label, 344).astype(np.uint8)
    selem = disk(2)
    cell_label = closing(cell_label, selem)
    cell_label = __fill_holes(cell_label)
    # this part is to use green channel, and extend cell label to green channel
    # benefit is to exclude cells clear on border but without nucleus
    sk = np.asarray(
        np.add(
            np.asarray(cell_label > 0, dtype=np.int8),
            np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8),
        )
        > 0,
        dtype=np.int8,
    )
    cell_label = segmentation.watershed(-distance, cell_label, mask=sk)
    cell_label = __fill_holes(cell_label)
    cell_label = np.asarray(cell_label > 0, dtype=np.uint8)
    cell_label = measure.label(cell_label)
    cell_label = remove_small_objects(cell_label, 344) # was 344
    cell_label = measure.label(cell_label)
    cell_label = np.asarray(cell_label, dtype=np.uint16)
    #nuclei_label = np.multiply(cell_label > 0, nuclei_label) > 0
    #nuclei_label = measure.label(nuclei_label)
    #nuclei_label = remove_small_objects(nuclei_label, 157)
    #nuclei_label = np.multiply(cell_label, nuclei_label > 0)

    return cell_label#nuclei_label, cell_label



In [ ]:
def get_predicton(mask,image,j,models,econding):
    
    mask = (mask == j).astype(int)[...,None]
    mask = np.float32(mask)
    mask=cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
    #image = blend2_image(imageID)
    result = mask * image
    result = get_cropped_resized_cell(result)
    img =  tf.reshape(result, (-1, IM_SIZE, IM_SIZE, 3))
    liste = sum([model.predict(img) for model in models]) / len(models)
    #print(liste)
        
    return ["%s" % i  + ' ' +  p.astype('str') + ' ' + econding  for i,p in enumerate(liste[0])]

In [ ]:

NUC_MODEL = "../input/hpacellsegmodel/nuclei-model.pth"
CELL_MODEL = "../input/hpacellsegmodel/cell-model.pth"


#NUC_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth"
#CELL_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth"

segmentator = cellsegmentator.CellSegmentator(
            NUC_MODEL,
            CELL_MODEL,
            scale_factor=0.25,
            device="cuda",
            padding=True,
            multi_channel_model=True,
        )

In [ ]:
def encode_mask(cell_mask,i):
    
    cell_mask = (cell_mask == i)#.astype(int)[...,None]
    
    #return '7'+' 1.0 ' + encode_binary_mask(ma.make_mask(cell_mask)).decode("utf-8")+ ' '
    #return ' ' + encode_binary_mask(ma.make_mask(cell_mask)).decode("utf-8")+ ' '
    return encode_binary_mask(cell_mask).decode("utf-8") + ' '
    

    
    
def pred_str2(imageID,size):
    
    

    directory = '../input/hpa-single-cell-image-classification/test/'



#im_string = '5f1af6b4-bb99-11e8-b2b9-ac1f6b6435d0'


  

    images = [
            [directory+ imageID + '_red.png'],
            [directory+ imageID + '_yellow.png'],
            [directory+ imageID + '_blue.png' ]
        ]

    #blue_batch = cv2.imread(directory+ imageID + '_blue.png', cv2.IMREAD_GRAYSCALE )
    #blue_batch = np.asarray(blue_batch)
    
    #red_batch = cv2.imread(directory+ imageID + '_blue.png', cv2.IMREAD_GRAYSCALE )
    #red_batch = np.asarray(red_batch)
        # For nuclei
        
    if os.path.isfile('../input/hpa-write-mask/' + imageID + '.npz'):
        
        cell_mask = np.load('../input/hpa-write-mask/' + imageID + '.npz')['label']
    else:
        nuc_segmentations = segmentator.pred_nuclei(images[2])
    #nuc_segmentations = segmentator.pred_nuclei(blue_batch)

        # For full cells
        cell_segmentations = segmentator.pred_cells(images)
    #cell_segmentations = cellsegmentor.pred_cells(rgb_batch, precombined=True)


        cell_mask = label_cell2(cv2.resize(nuc_segmentations[0],(512,512)), cv2.resize(cell_segmentations[0],(512,512)))
        #cell_mask = resize(cell_mask,(size,size),preserve_range=True)
        #cell_mask = label_cell3(nuc_segmentations[0], cell_segmentations[0])
        cell_mask = cv2.resize(cell_mask,(size,size),interpolation=cv2.INTER_NEAREST)
        
    
    
    
    #s = [encode_mask(cell_mask,i)  for i in range(1,np.amax(cell_mask)+1)]
    
    blend_image = blend2_image(imageID)
    
    #s = [encode_mask(cell_mask,i).join(["%s" % p[0]  + ' ' +  np.round(p[1],3).astype('str')  for p in enumerate(get_predicton(cell_mask,blend_image,i,model))]) for i in range(1,np.amax(cell_mask)+1)]
    
    s = []
    
    for i in range(1,np.amax(cell_mask)+1):
        
        #mask2 = np.copy(cell_mask)
        #mask2[mask2 > i] = 0
        #mask2[mask2 < i] = 0
        #mask2 = resize(mask2,(size,size))
        #mask2[mask2 > 0] = 1
        
        
        encoding = encode_mask(cell_mask,i)
        
        #liste =  ["%s" % p[0]  + ' ' +  p[1].astype('str') + ' ' + encoding 
        #          for p in enumerate(get_predicton(cell_mask,blend_image,i,model))]
        
        #output = [i + ' ' + string for i in liste]
        #output = ''.join(liste)
        output = ''.join(get_predicton(cell_mask,blend_image,i,models,encoding))
        s.append(output)
    #    s.append(''.join(output))
    
    #for i in range(1,np.amax(cell_mask)+1):
   
    
    #    test_mask = cell_mask
    #    test_mask = (test_mask == i).astype(int)[...,None]
    #    s.append('0'+' 1.0 ' + encode_binary_mask(ma.make_mask(test_mask)).decode("utf-8")+ ' ')
    return ''.join(s)[:-1]

def getmask(imageID,size):
    
    

    directory = '../input/hpa-single-cell-image-classification/test/'



#im_string = '5f1af6b4-bb99-11e8-b2b9-ac1f6b6435d0'


  

    images = [
            [directory+ imageID + '_red.png'],
            [directory+ imageID + '_yellow.png'],
            [directory+ imageID + '_blue.png' ]
        ]

    #blue_batch = cv2.imread(directory+ imageID + '_blue.png', cv2.IMREAD_GRAYSCALE )
    #blue_batch = np.asarray(blue_batch)
    
    #red_batch = cv2.imread(directory+ imageID + '_blue.png', cv2.IMREAD_GRAYSCALE )
    #red_batch = np.asarray(red_batch)
        # For nuclei
        
    if os.path.isfile('../input/hpa-write-mask/' + imageID + '.npz'):
        
        cell_mask = np.load('../input/hpa-write-mask/' + imageID + '.npz')['label']
    else:
        nuc_segmentations = segmentator.pred_nuclei(images[2])
    #nuc_segmentations = segmentator.pred_nuclei(blue_batch)

        # For full cells
        cell_segmentations = segmentator.pred_cells(images)
    #cell_segmentations = cellsegmentor.pred_cells(rgb_batch, precombined=True)


        cell_mask = label_cell2(cv2.resize(nuc_segmentations[0],(512,512)), cv2.resize(cell_segmentations[0],(512,512)))
        #cell_mask = cell_mask * 10000
        #cell_mask = cv2.resize(cell_mask,(size,size))
        #cell_mask = cell_mask / 10000
        #cell_mask = cell_mask.astype(int)
        
        #kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,8))
        #cell_mask = cv2.morphologyEx(cell_mask, cv2.MORPH_CLOSE, kernel)
        #cell_mask = remove_small_objects(cell_mask, 1000)
        #cell_mask = label_cell3(nuc_segmentations[0], cell_segmentations[0])
        
    return cell_mask

In [ ]:
df_imlist

In [ ]:
#for i, row in df_imlist.iterrows():
#    print(np.amax(getmask(row['ID'],row['ImageWidth'])))

In [ ]:
start = time.time()

df_imlist['PredictionString'] =  df_imlist.apply(
    lambda row: pred_str2(row['ID'],row['ImageWidth']),
    axis=1
)




end = time.time()

print((end - start)/3)

In [ ]:
df_imlist['PredictionString'][0]

# Image level Models

In [ ]:
sub_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')

if debug:
    sub_df = sub_df[:3]
else:
    sub_df = sub_df

def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img

    def decode_with_labels(path, label):
        return decode(path), label

    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img

    def augment_with_labels(img, label):
        return augment(img), label

    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)

    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)

    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)

    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)

    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)

    return dset

COMPETITION_NAME = "hpa-single-cell-image-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 512)

load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
#sub_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
#sub_df = ss_df.copy()

sub_df = sub_df.drop(sub_df.columns[1:],axis=1)

for i in range(19):
    sub_df[f'{i}'] = pd.Series(np.zeros(sub_df.shape[0]))


test_paths = load_dir + "/test/" + sub_df['ID'] + '_green.png'
# Get the multi-labels
label_cols = sub_df.columns[1:]

test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[8], IMSIZE[8]))
dtest = build_dataset(
    test_paths, bsize=BATCH_SIZE, repeat=False, 
    shuffle=False, augment=True, cache=False,
    decode_fn=test_decoder
)

with strategy.scope():
    models_full = []

    model_full = tf.keras.models.load_model(
        '../input/fullimageeff5/HPA_model_effect5_RS10_full.h5',custom_objects={"FixedDropout": keras.layers.Dropout,"macro_f1" : macro_f1}
        )

    model_full1 = tf.keras.models.load_model(
        '../input/fullimageeffe5rs11/HPA_model_effect5_RS11_full.h5',custom_objects={"FixedDropout": keras.layers.Dropout,"macro_f1" : macro_f1}
        )
    
    model_full2 = tf.keras.models.load_model(
        '../input/fulldensrs12/HPA_model_effect7_RS12_full.h5',custom_objects={"FixedDropout": keras.layers.Dropout,"macro_f1" : macro_f1}
        )

 
    
    
    models_full.append(model_full)
    models_full.append(model_full1)
    models_full.append(model_full2)
    #models_full.append(model_full3)

#model.summary()
sub_df[label_cols] = sum([model.predict(dtest, verbose=1) for model in models_full]) / len(models_full)

sub_df.head()

ss_df = pd.merge(sub_df, df_imlist, on = 'ID', how = 'left')

for i in range(ss_df.shape[0]):
    if ss_df.loc[i,'PredictionString'] == '0 1 eNoLCAgIMAEABJkBdQ==':
        continue
    a = ss_df.loc[i,'PredictionString']
    b = a.split()
    for j in range(int(len(a.split())/3)):
        for k in range(19):
            if int(b[0 + 3 * j]) == k:

                c = b[0 + 3 * j + 1]               
                b[0 + 3 * j + 1] = str(ss_df.loc[i,f'{k}'] * 0.4 + float(c) * 0.6)# * 0.9 + float(c) * 0.1

    ss_df.loc[i,'PredictionString'] = ' '.join(b)

ss_df = ss_df[['ID','ImageWidth','ImageHeight','PredictionString']]
ss_df.to_csv('submission.csv',index = False)



In [ ]:
ss_df

In [ ]:
df_imlist

In [ ]:
COMPETITION_NAME = "hpa-single-cell-image-classification"
load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
sub_df = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
    #sub_df = ss_df.copy()

sub_df = sub_df.drop(sub_df.columns[1:],axis=1)
sub_df